 # Función 6 ML

1. **def recomendacion_juego( id de producto )**:
- DataFrame a utilizar: `df_EDA_steam_for_ML`
- DataFrame a utilizar: `df_EDA_reviws_for_ML`
- Utiliza la columna  `id` para saber el id del juego.
- Utiliza la columna `app_name` para saber el nombre del juego.
- Utiliza la columna `used_id` para saber el id del usuario.
- Utiliza la columna `sentiment` para saber el sentimiento del juego.
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Importando las librerías necesarias.

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt


In [11]:
df_steam = pd.read_csv('df_EDA_steam_for_ML.csv')
df_steam = df_steam [['id','app_name']]
df_steam

,id,app_name
0,761140.0,Lost Summoner Kitty
1,761140.0,Lost Summoner Kitty
2,761140.0,Lost Summoner Kitty
3,761140.0,Lost Summoner Kitty
4,761140.0,Lost Summoner Kitty
...,...,...
1924627,658870.0,EXIT 2 - Directions
1924628,658870.0,EXIT 2 - Directions
1924629,658870.0,EXIT 2 - Directions
1924630,658870.0,EXIT 2 - Directions


In [12]:
df_reviews = pd.read_csv('df_EDA_reviws_for_ML.csv')
df_reviews = df_reviews[['user_id', 'id', 'sentiment']]
df_reviews

,user_id,id,sentiment
0,76561197970982479,1250.0,2
1,76561197970982479,22200.0,2
2,76561197970982479,43110.0,2
3,js41637,251610.0,2
4,js41637,227300.0,0
...,...,...,...
48466,wayfeng,730.0,1
48467,76561198251004808,253980.0,2
48468,72947282842,730.0,0
48469,ApxLGhost,730.0,2


In [13]:
df_reviews = df_reviews[['user_id', 'id', 'sentiment']].merge(df_steam[['id', 'app_name']], on='id', how='left')

In [14]:
df_reviews

,user_id,id,sentiment,app_name
0,76561197970982479,1250.0,2,Killing Floor
1,76561197970982479,1250.0,2,Killing Floor
2,76561197970982479,1250.0,2,Killing Floor
3,76561197970982479,1250.0,2,Killing Floor
4,76561197970982479,1250.0,2,Killing Floor
...,...,...,...,...
13495767,ApxLGhost,730.0,2,Counter-Strike: Global Offensive
13495768,ApxLGhost,730.0,2,Counter-Strike: Global Offensive
13495769,ApxLGhost,730.0,2,Counter-Strike: Global Offensive
13495770,ApxLGhost,730.0,2,Counter-Strike: Global Offensive


In [15]:
df_reviews = df_reviews.drop_duplicates()

In [16]:
df_reviews = df_reviews.rename(columns={'id': 'item_id'})


In [17]:
df_reviews = df_reviews.dropna(subset=['app_name'])

In [18]:
df_reviews['app_name'] = df_reviews['app_name'].astype(str)

In [19]:
df_reviews 

,user_id,item_id,sentiment,app_name
0,76561197970982479,1250.0,2,Killing Floor
200,76561197970982479,22200.0,2,Zeno Clash
306,js41637,227300.0,0,Euro Truck Simulator 2
546,js41637,239030.0,0,"Papers, Please"
786,evcentric,370360.0,2,TIS-100
...,...,...,...,...
13495080,76561198239215706,730.0,1,Counter-Strike: Global Offensive
13495240,wayfeng,730.0,1,Counter-Strike: Global Offensive
13495400,76561198251004808,253980.0,2,Enclave
13495451,72947282842,730.0,0,Counter-Strike: Global Offensive


In [20]:
df = df_reviews
df.to_csv('df_ML.csv')


Filtrado Colaborativo para el sistema de recomendación user-item
Uso la biblioteca Surprise para entrenar el modelo.  

In [21]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy, SVD
from surprise.model_selection import train_test_split
import joblib
from surprise import SVD
from surprise.model_selection import cross_validate

In [42]:
df = pd.read_csv('df_ML.csv')
df

,Unnamed: 0,user_id,item_id,sentiment,app_name
0,0,76561197970982479,1250.0,2,Killing Floor
1,200,76561197970982479,22200.0,2,Zeno Clash
2,306,js41637,227300.0,0,Euro Truck Simulator 2
3,546,js41637,239030.0,0,"Papers, Please"
4,786,evcentric,370360.0,2,TIS-100
...,...,...,...,...,...
40224,13495080,76561198239215706,730.0,1,Counter-Strike: Global Offensive
40225,13495240,wayfeng,730.0,1,Counter-Strike: Global Offensive
40226,13495400,76561198251004808,253980.0,2,Enclave
40227,13495451,72947282842,730.0,0,Counter-Strike: Global Offensive


Instanciamos la clase Reader() y definimos la escala de sentiment

In [23]:
reader = Reader(rating_scale=(df['sentiment'].min(), df['sentiment'].max()))

Abro el Dataset usando Dataset.load_from_df

In [24]:
data = Dataset.load_from_df(df[['user_id', 'item_id', 'sentiment']], reader)

#data = Dataset.load_from_file(file_path, reader=reader) otra opcion


Separo el data con tran_test_split, dejando un 30% de la data para test.

In [25]:
trainset, testset = train_test_split(data, test_size=.30)

Entrenamiento Opcion I

In [26]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)



RMSE: 0.7997


0.7997231431652781

Entrenamiento Opcion II

evaluación del rendimiento de un algoritmo  SVD (Singular Value Decomposition) en un proceso de validación cruzada con 3 divisiones (folds). La validación cruzada se utiliza comúnmente en machine learning para evaluar el rendimiento de un modelo de manera más robusta

In [27]:
#creo svd
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8094  0.8060  0.7950  0.8035  0.0062  
MAE (testset)     0.7182  0.7128  0.7074  0.7128  0.0044  
Fit time          0.23    0.24    0.20    0.22    0.02    
Test time         0.09    0.14    0.08    0.10    0.02    


{'test_rmse': array([0.80942084, 0.80602168, 0.79498462]),
 'test_mae': array([0.71818995, 0.71279259, 0.70737906]),
 'fit_time': (0.2291407585144043, 0.23771119117736816, 0.19841623306274414),
 'test_time': (0.08575892448425293, 0.13568758964538574, 0.08106732368469238)}

Entrenamiento del modelo SVD


Generación de predicciones de calificación/score.
    
    Predecimos sobre el testset, ya que no lo conoce el modelo creado SVD

In [28]:
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [29]:
predictions = svd.test(testset)


In [30]:
print(predictions [1])
print(predictions [100])

user: Bogers69   item: 207610.0   r_ui = 1.00   est = 1.25   {'was_impossible': False}
user: Pendekar   item: 286360.0   r_ui = 2.00   est = 1.48   {'was_impossible': False}


Guardamos el modelo desarrollado


In [31]:


joblib.dump(algo, 'modelo_svd_I.pkl')


# Suponiendo que 'modelo_svd' es tu modelo entrenado SVD
joblib.dump(svd, 'modelo_svd_II.pkl')



['modelo_svd_II.pkl']

Esta funcion devuelve los 5 videojuegos que le gustaran al usuario ingresado del DataFrame user_reviews de Steam Games

In [44]:
def  recomendacion_usuario(user_id:str):
    modelo_svd = joblib.load('modelo_svd_I.pkl')
    top=5

    # Obtener las películas que el usuario no ha visto aún
    games_play = set(df[df['user_id'] == user_id]['item_id'])
    games_all = set(df['item_id'])
    games_unplay = list(games_all - games_play)

    # Obtener las recomendaciones
    predicted_ratings = [modelo_svd.predict(user_id, item_id).est for item_id in games_unplay]

    # Ordenar los juegos según su predicción de rating
    games_rating = list(zip(games_unplay, predicted_ratings))
    games_rating.sort(key=lambda x: x[1], reverse=True)

    # Obtener los títulos de los juegos recomendadas
    recommended_games = games_rating[:top]
    recommended_games = [df[df['item_id'] == item_id]['app_name'].iloc[0] for item_id, _ in recommended_games]

    return {
        "recomended_games": recommended_games,
        "user_id": user_id,
        "top": 5
    }

In [33]:
recomendacion_usuario(76561198069536565)

{'recomended_games': ['Halo: Spartan Assault',
  "King Arthur's Gold",
  'Tales from the Borderlands',
  'Wolfenstein: The New Order',
  'RIFT'],
 'user_id': 76561198069536565,
 'top': 5}

In [48]:
df_subset = df.head(500)

# Crear una lista para almacenar las recomendaciones de cada usuario
dicc = []

# Iterar sobre cada user_id en el DataFrame subset
for user_id in df_subset['user_id']:
    # Obtener las recomendaciones para el usuario actual
    recomendaciones_usuario = recomendacion_usuario(user_id)
    
    # Agregar las recomendaciones al diccionario
    dicc.append({
        'user_id': user_id,
        'recomendaciones': recomendaciones_usuario['recomended_games']
    })

# Convertir la lista de diccionarios en un DataFrame
df_recomendaciones = pd.DataFrame(dicc)
df_recomendaciones

# Mostrar el DataFrame de recomendaciones
print(df_recomendaciones)

               user_id                                    recomendaciones
0    76561197970982479  [Wolfenstein: The New Order, Halo: Spartan Ass...
1    76561197970982479  [Wolfenstein: The New Order, Halo: Spartan Ass...
2              js41637  [Burnout Paradise: The Ultimate Box, Mark of t...
3              js41637  [Burnout Paradise: The Ultimate Box, Mark of t...
4            evcentric  [Eternal Senia, RIFT, Sid Meier's Civilization...
..                 ...                                                ...
495  76561197974500703  [Wolfenstein: The New Order, RIFT, King Arthur...
496  76561197974500703  [Wolfenstein: The New Order, RIFT, King Arthur...
497  76561197974500703  [Wolfenstein: The New Order, RIFT, King Arthur...
498  76561197974500703  [Wolfenstein: The New Order, RIFT, King Arthur...
499  76561197974500703  [Wolfenstein: The New Order, RIFT, King Arthur...

[500 rows x 2 columns]


In [1]:
df_recomendaciones.to_csv('ML_recommendations500.csv')

NameError: name 'df_recomendaciones' is not defined

Haciendo un ejercicio

In [6]:
import pandas as pd
def darjuegosFavoritos(user_id:str):
    df_E=pd.read_csv('ML_recommendations500.csv')
    recomendaciones_usuario = df_E.loc[df_E['user_id'] == user_id, 'recomendaciones']
    return str(recomendaciones_usuario)

In [7]:
darjuegosFavoritos('76561197974500703')

'495    [\'Wolfenstein: The New Order\', \'RIFT\', "King A...\n496    [\'Wolfenstein: The New Order\', \'RIFT\', "King A...\n497    [\'Wolfenstein: The New Order\', \'RIFT\', "King A...\n498    [\'Wolfenstein: The New Order\', \'RIFT\', "King A...\n499    [\'Wolfenstein: The New Order\', \'RIFT\', "King A...\nName: recomendaciones, dtype: object'